In [7]:
from pathlib import Path
import sys
import os
from pydantic import SecretStr

current_folder = Path.cwd()
src_root = current_folder.parent
project_root = src_root.parent
sys.path.insert(0, str(project_root))


from src.config import QDRANT_HOST, GEMINI_API_KEY

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain_qdrant import QdrantVectorStore, RetrievalMode, FastEmbedSparse

print("Imports successful!")

Imports successful!


### Configuration

In [8]:
COLLECTION_NAME = "dekkingen"
EMBEDDING_MODEL_NAME = "models/embedding-001"
COLLECTION_NAME = "dekkingen"
EMBEDDING_MODEL_NAME = "models/embedding-001"  # Google's embedding model
EMBEDDING_DIMENSION = 768  # embedding-001 produces 768-dimensional vectors
# Sparse embedding configuration for hybrid retrieval
SPARSE_MODEL_NAME = "Qdrant/bm25"  # BM25 sparse embeddings
DENSE_VECTOR_NAME = "dense"  # Name for dense vectors in Qdrant
SPARSE_VECTOR_NAME = "sparse"  # Name for sparse vectors in Qdrant

embeddings = GoogleGenerativeAIEmbeddings(
            model=EMBEDDING_MODEL_NAME,
            google_api_key=SecretStr(GEMINI_API_KEY) if GEMINI_API_KEY else None
        )

sparse_embeddings = FastEmbedSparse(model_name=SPARSE_MODEL_NAME)

Fetching 18 files: 100%|██████████| 18/18 [00:00<00:00, 34.99it/s]


### Retrieving

In [9]:
client = QdrantClient(host=QDRANT_HOST)

In [20]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="dekkingen",
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    vector_name="dense",
    sparse_vector_name="sparse",
)

In [21]:
query = "wat kan ik verwachten voor zwangerschapsdekking in verenigde staten?"

dense_result = vector_store.similarity_search_with_score(
    query,
    k=5,
)


In [22]:
for (doc, score) in dense_result:
    print(doc.metadata)
    print(doc.page_content)
    print(f"Score: {score}\n")

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T20:19:55.751488', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Zwangerschap en bevalling', 'header_5': 'Bevalling (niet in VS)', '_id': '98d00a1d-4fbc-4f96-8395-102e2f172d67', '_collection_name': 'dekkingen'}
#### Zwangerschap en bevalling  
##### Bevalling (niet in VS)  
Dit betreft de vergoeding van de kosten voor een bevalling die plaatsvindt buiten de Verenigde Staten. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.
Score: 0.8216406

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T20:19:55.751488', 'file

In [10]:
for (doc, score) in result:
    print(doc.metadata)
    print(doc.page_content)
    print(f"Score: {score}\n")
    

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T16:08:12.871179', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md', 'header_1': 'Expat Pakket Individueel - Premie- en dekkingsoverzicht 2025', 'header_2': '2. Ziektekosten', 'header_3': '2.1 Verzekerde rubrieken ziektekosten', 'header_4': 'Zwangerschap en bevalling', 'header_5': 'Sterilisatie', '_id': '50efb225-8aab-4cc3-bb7c-5d034896a868', '_collection_name': 'dekkingen'}
##### Sterilisatie  
Dit betreft de vergoeding voor een medische ingreep die resulteert in permanente onvruchtbaarheid. Dit is niet gedekt onder het Standaard plan. Voor het Optimaal plan is dit 100% gedekt. Voor het Excellent plan is dit 100% gedekt.
Score: 0.8250388

{'document_name': 'goudse_dekking.md', 'document_id': '66cf5937e9872a08', 'company': 'Goudse', 'ingestion_timestamp': '2025-10-23T16:08:12.871179', 'filepath': '/app/data/documents/dekkingen/goudse_dekking.md',